<a href="https://colab.research.google.com/github/abhiram622/gitdemo11/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tokenizers
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from transformers import AutoTokenizer


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

In [ ]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

In [ ]:
def get_vocab_size(tokenizer) -> int:
    vocab = tokenizer.get_vocab_size()
    return vocab

In [ ]:
import torch
torch.manual_seed(3647)

block_size = 256
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
vocab_size = get_vocab_size(tokenizer)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from typing import Optional, Tuple
import torch
import torch.nn as nn
from torch.nn import functional as F


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size: int) -> None:
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(
            torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        _, T, _ = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x)  # (B,T,hs)
        # compute attention scores ("affinities")
        # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        weights = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        weights = weights.masked_fill(
            self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        weights = F.softmax(weights, dim=-1)  # (B, T, T)
        weights = self.dropout(weights)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,hs)
        out = weights @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads: int, head_size: int) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.projection(out))
        return out

**bold text**

**3.BLOCK**

In [ ]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd: int) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd: int, n_head: int) -> None:
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.self_attention = MultiHeadAttention(n_head, head_size)
        self.feed_forward = FeedForward(n_embd)
        self.layer_norm_1 = nn.LayerNorm(n_embd)
        self.layer_norm_2 = nn.LayerNorm(n_embd)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.self_attention(self.layer_norm_1(x))
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [58]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size, block_size, n_embd, n_head, n_layer, dropout, device):
        super().__init__()

        self.device = device

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head) for _ in range(n_layer)]
        )

        self.final_layer_norm = nn.LayerNorm(n_embd)
        self.final_linear_layer = nn.Linear(n_embd, vocab_size)

        # Initialize weights
        self.apply(self._init_weights)

    # ⭐⭐⭐ THIS MUST BE INSIDE THE CLASS ⭐⭐⭐
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, input_tokens, targets=None):
        B, T = input_tokens.shape

        token_embedding = self.token_embedding_table(input_tokens)
        positional_embedding = self.position_embedding_table(
            torch.arange(T, device=self.device)
        )

        x = token_embedding + positional_embedding
        x = self.blocks(x)
        x = self.final_layer_norm(x)
        logits = self.final_linear_layer(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, input_tokens, max_new_tokens):
        for _ in range(max_new_tokens):
            cropped = input_tokens[:, -block_size:]
            logits, _ = self(cropped)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            input_tokens = torch.cat((input_tokens, next_token), dim=1)
        return input_tokens


**5. Parameters & dummy input**

In [59]:
model = GPTLanguageModel(
    vocab_size=vocab_size,
    block_size=block_size,
    n_embd=n_embd,
    n_head=n_head,
    n_layer=n_layer,
    dropout=dropout,
    device=device
).to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

11.004249 M parameters


In [60]:
batch_size = 1
seq_length = 6
x = torch.randint(0, vocab_size, (batch_size, seq_length))
x = x.to(device)

# Test the token embedding table directly
print(f"Input tensor x shape: {x.shape}, dtype: {x.dtype}, device: {x.device}")
print(f"Embedding table weight shape: {model.token_embedding_table.weight.shape}, dtype: {model.token_embedding_table.weight.dtype}, device: {model.token_embedding_table.weight.device}")

try:
    test_embedding = model.token_embedding_table(x)
    print(f"Direct embedding test successful. Output shape: {test_embedding.shape}")
except Exception as e:
    print(f"Direct embedding test failed: {e}")

# Original model forward pass
logits, loss = model(x)
print(logits.shape, loss)

Input tensor x shape: torch.Size([1, 6]), dtype: torch.int64, device: cpu
Embedding table weight shape: torch.Size([345, 384]), dtype: torch.float32, device: cpu
Direct embedding test successful. Output shape: torch.Size([1, 6, 384])
torch.Size([1, 6, 345]) None


In [61]:
def print_model_structure(model: torch.nn.Module, indent: str = '') -> None:
    """
    Custom function to print model structure in a hierarchical format
    """
    for name, child in model.named_children():
        params = sum(p.numel() for p in child.parameters())
        print(f"{indent}├─ {name}: {child.__class__.__name__} ({params:,} parameters)")
        print_model_structure(child, indent + '│  ')


print_model_structure(model)

├─ token_embedding_table: Embedding (132,480 parameters)
├─ position_embedding_table: Embedding (98,304 parameters)
├─ blocks: Sequential (10,639,872 parameters)
│  ├─ 0: Block (1,773,312 parameters)
│  │  ├─ self_attention: MultiHeadAttention (590,208 parameters)
│  │  │  ├─ heads: ModuleList (442,368 parameters)
│  │  │  │  ├─ 0: Head (73,728 parameters)
│  │  │  │  │  ├─ key: Linear (24,576 parameters)
│  │  │  │  │  ├─ query: Linear (24,576 parameters)
│  │  │  │  │  ├─ value: Linear (24,576 parameters)
│  │  │  │  │  ├─ dropout: Dropout (0 parameters)
│  │  │  │  ├─ 1: Head (73,728 parameters)
│  │  │  │  │  ├─ key: Linear (24,576 parameters)
│  │  │  │  │  ├─ query: Linear (24,576 parameters)
│  │  │  │  │  ├─ value: Linear (24,576 parameters)
│  │  │  │  │  ├─ dropout: Dropout (0 parameters)
│  │  │  │  ├─ 2: Head (73,728 parameters)
│  │  │  │  │  ├─ key: Linear (24,576 parameters)
│  │  │  │  │  ├─ query: Linear (24,576 parameters)
│  │  │  │  │  ├─ value: Linear (24,576 param

In [62]:
import pandas as pd


def get_model_stats(model: torch.nn.Module) -> pd.DataFrame:
    """
    Create a DataFrame with detailed layer statistics
    """
    stats = []
    for name, module in model.named_modules():
        if len(list(module.children())) == 0:  # Only leaf modules
            params = sum(p.numel() for p in module.parameters())
            stats.append({
                'Layer Name': name,
                'Type': module.__class__.__name__,
                'Parameters': params,
                'Trainable': sum(p.numel() for p in module.parameters() if p.requires_grad)
            })
    return pd.DataFrame(stats)


stats_df = get_model_stats(model)
stats_df

,Layer Name,Type,Parameters,Trainable
0,token_embedding_table,Embedding,132480,132480
1,position_embedding_table,Embedding,98304,98304
2,blocks.0.self_attention.heads.0.key,Linear,24576,24576
3,blocks.0.self_attention.heads.0.query,Linear,24576,24576
4,blocks.0.self_attention.heads.0.value,Linear,24576,24576
...,...,...,...,...
191,blocks.5.feed_forward.net.3,Dropout,0,0
192,blocks.5.layer_norm_1,LayerNorm,768,768
193,blocks.5.layer_norm_2,LayerNorm,768,768
194,final_layer_norm,LayerNorm,768,768
